In [1]:
import os
print(os.getcwd())#显示当前路径
 
os.chdir('D:/bio/metirc/scib-main')#更改路径，''里面为更改的路径

print(os.getcwd())#显示当前路径

os.environ['R_HOME'] = "E:/R/R-4.2.1/"
os.environ['R_USER'] = "D:/anaconda/envs/index/Lib/site-packages/rpy2"

D:\bio\jupyter_code\index
D:\bio\metirc\scib-main


In [2]:
import index

# import importlib
# importlib.reload(index)

In [3]:
import scib
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad 

import esda
from esda.moran import Moran, Moran_Local
from esda.geary import Geary
from esda.geary_local import Geary_Local
from esda.getisord import G, G_Local

In [ ]:
# print(STUtility_coor_pd.shape)
# print(adata_pre.obs_names[:5])
# print(STUtility_coor_pd.head())

In [10]:
import scipy

def load_data(start,end):
    adata_list = []
    for i in range(start,end):
        input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'
        adata_temp = sc.read_visium(input_dir)
        adata_temp.var_names_make_unique()
        adata_temp.obs_names_make_unique()
        adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
        adata_temp.obs['batch'] = adata_label['batch'].astype("category")
        adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
        adata_list.append(adata_temp)
    adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(start,end)])
    return adata_pre


methods = ['STUtility', 'moscot_affine', 'moscot_warp', 'STalign', 'paste', 'SPACEL', 'SPIRAL']

for method in methods:
    if method == 'STUtility':
        continue
        adata_pre = load_data(151673, 151677)
        STUtility_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STUtility/Donor3-STutil-coords.csv", sep=" ")
        STUtility_coord_list = []
        STUtility_coor_pd.rename(index=lambda s: s.split('_',1)[0] + '-' + str(int(s.split('_',1)[1]) + 151672), inplace=True)
        adatas = adata_pre[STUtility_coor_pd.index.tolist(), :]
        adatas.obsm['spatial_aligned'] = STUtility_coor_pd[['align_x', 'align_y']].values
        adatas.obsm['spatial_origin'] = STUtility_coor_pd[['x', 'y']].values
    if method == 'moscot_affine':
        continue
        adata_pre = load_data(151673, 151677)
        moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_affine_donor3.txt", header = None)
        adatas = adata_pre
        adatas.obsm['spatial_aligned'] = moscot_coor_pd.values
        adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
    if method == 'moscot_warp':
        continue
        adata_pre = load_data(151673, 151677)
        moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_warp_donor3.txt", header = None)
        adatas = adata_pre
        adatas.obsm['spatial_aligned'] = moscot_coor_pd.values
        adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
    if method == 'STalign':
        continue
        adata_list = []
        for i in range(151673, 151677):
            input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'
            adata_temp = sc.read_visium(input_dir)
            adata_temp.var_names_make_unique()
            adata_temp.obs_names_make_unique()
            adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
            adata_temp.obs['batch'] = adata_label['batch'].astype("category")
            adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
            if i != 151676:
                STalign_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/STalign/'+str(i)+'_aligned_to_'+str(i+1)+'.csv')
                STalign_coor_pd.index = STalign_coor_pd['0']
                adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
                adata_temp.obsm['spatial_aligned'] = STalign_coor_pd[['aligned_y', 'aligned_x']].values
            else:
                adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
                adata_temp.obsm['spatial_aligned'] = adata_temp.obsm['spatial'].copy()
            adata_list.append(adata_temp)
        adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151673,151677)])
        adatas = adata_pre
        del adata_list
        del adata_temp
    if method == 'paste':
        continue
        adata_list = []
        for i in range(151673,151677):
            input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'
            adata_temp = sc.read_visium(input_dir)
            adata_temp.var_names_make_unique()
            adata_temp.obs_names_make_unique()
            adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
            adata_temp.obs['batch'] = adata_label['batch'].astype("category")
            adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
            paste_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/PASTE/'+str(i)+'.csv')
            paste_coor_pd.index = paste_coor_pd['spot']
            paste_coor_pd.rename(index=lambda s: s.split('.',1)[0], inplace=True)
            adata_temp = adata_temp[paste_coor_pd.index.tolist(), :]
            adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
            adata_temp.obsm['spatial_aligned'] = paste_coor_pd[['aligned_x', 'aligned_y']].values
            adata_list.append(adata_temp)
        adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151673,151677)])
        print(adata_pre)
        adatas = adata_pre
        del adata_temp
        del adata_list
    if method == 'SPACEL':
        adata_pre = load_data(151673, 151677)
        SPACEL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPACEL/donor3_aligned_coordinates.csv")
        adatas = adata_pre
        adatas.obsm['spatial_aligned'] = SPACEL_coor_pd[['X','Y']].values
        adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
    if method == 'SPIRAL':
        adata_pre = load_data(151673, 151677)
        SPIRAL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/output/spiral/new_coord_151673_151674_151675_151676_modify.csv")
        SPIRAL_coor_pd.index = SPIRAL_coor_pd['Unnamed: 0']
        SPIRAL_coor_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
        adatas = adata_pre[SPIRAL_coor_pd.index.tolist(), :]
        adatas.obsm['spatial_aligned'] = SPIRAL_coor_pd[['x', 'y']].values
        adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
        
    ###SCC
    # source
    adatas.obsm['spatial'] = adatas.obsm['spatial_origin'].copy()
    knn = 6
    g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
    scs = index.spatial_coherence_score(g, node_dict, 100)
    adatas.uns['scc_origin'] = scs
    # align
    adatas.obsm['spatial'] = adatas.obsm['spatial_aligned'].copy()
    knn = 6
    g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
    scs = index.spatial_coherence_score(g, node_dict, 100)
    adatas.uns['scc_aligned'] = scs
    
    groups = adatas.obs.groupby("batch").indices
    adata_list = [adatas[i] for i in groups.values()]
    
    ###moran's I & geary's C
    for i in range(4):
        batch = list(groups.keys())[i].split('_')[1]
        input_dir = 'G:/dataset/1_DLPFC/input/' + batch + '/'
        SVGs = pd.read_csv(input_dir + 'SPARKX.csv', index_col=0)
        SVGs = SVGs.sort_values(by="adjustedPval",ascending=True)
        # source
        moranI_dict = {}
        gearyC_dict = {}
        sc.pp.neighbors(adata_list[i], use_rep='spatial_origin')
        num = [1,10,50,100,500,1000]
        for SVGnum in num:
            print(SVGnum)
            if SVGnum > SVGs.shape[0]:
                break
            moranI = np.mean( sc.metrics.morans_i(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
            moranI_dict["moranI_"+str(SVGnum)] = moranI
            gearyC = np.mean( sc.metrics.gearys_c(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
            gearyC_dict["gearyC_"+str(SVGnum)] = gearyC
        # align
        moranI_align_dict = {}
        gearyC_align_dict = {}
        sc.pp.neighbors(adata_list[i], use_rep='spatial_aligned')
        num = [1,10,50,100,500,1000]
        for SVGnum in num:
            print(SVGnum)
            if SVGnum > SVGs.shape[0]:
                break
            moranI = np.mean( sc.metrics.morans_i(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
            moranI_align_dict["moranI_"+str(SVGnum)] = moranI
            gearyC = np.mean( sc.metrics.gearys_c(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
            gearyC_align_dict["gearyC_"+str(SVGnum)] = gearyC
        adata_list[i].uns['moranI'] = moranI_dict
        adata_list[i].uns['moranI_align'] = moranI_align_dict
        adata_list[i].uns['gearyC'] = gearyC_dict
        adata_list[i].uns['gearyC_align'] = gearyC_align_dict
    
    ### SSIM & PCC
    for i in range(len(adata_list)):
        adata_list[i].obsm['spatial_aligned'] = pd.DataFrame(adata_list[i].obsm['spatial_aligned'], index=adata_list[i].obs_names)
        adata_list[i].obsm['spatial_aligned'].columns = ['X', 'Y']
    # each slice is rasterized on the overlap of the previous slice
    for i in range(1, len(adata_list)):
        x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i - 1].obsm['spatial_aligned'].max().X]))
        y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i - 1].obsm['spatial_aligned'].max().Y]))
        x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i - 1].obsm['spatial_aligned'].min().X]))
        y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i - 1].obsm['spatial_aligned'].min().Y]))
        adata_list[i].uns['grid_prop_withbefore'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                             grid_max=[x_max, y_max], grid_min=[x_min, y_min])
    # each slice is rasterized on the overlap of the next slice
    for i in range(0, len(adata_list)-1):
        x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i + 1].obsm['spatial_aligned'].max().X]))
        y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i + 1].obsm['spatial_aligned'].max().Y]))
        x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i + 1].obsm['spatial_aligned'].min().X]))
        y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i + 1].obsm['spatial_aligned'].min().Y]))
        adata_list[i].uns['grid_prop_withnext'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                           grid_max=[x_max, y_max], grid_min=[x_min, y_min])
    # calculate SSIM on the overlap of each two adjacent slices
    for i in range(1, len(adata_list)):
        grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
        grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
        col = set(grid_a.columns) & set(grid_b.columns)
        grid_a = grid_a.loc[:, col]
        grid_b = grid_b.loc[:, col]
        ssim_list = []
        for n in range(grid_a.shape[0]):
            ssim_list.append(index.ssim(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values))
        ssim_list = np.array(ssim_list)
        ssim_list[grid_b.sum(1) == 0] = np.inf
        ssim_list[pd.isna(pd.Series(ssim_list))] = 0
        adata_list[i].uns['ssim'] = ssim_list
    ssim_compare = []
    for i in range(1, len(adata_list)):
        ssim_compare.append([adata_list[i].uns['ssim'][adata_list[i].uns['ssim'] != np.inf].mean(), i, method])
    ssim_compare = pd.DataFrame(ssim_compare).fillna(0)
    ssim_compare.columns = ['ssim', 'slice', 'method']
    print(ssim_compare)
    
    # calculate PCC on the overlap of each two adjacent slices
    for i in range(1, len(adata_list)):
        grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
        grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
        col = set(grid_a.columns) & set(grid_b.columns)
        grid_a = grid_a.loc[:, col]
        grid_b = grid_b.loc[:, col]
        pcc_list = []
        for n in range(grid_a.shape[0]):
            pcc_list.append(scipy.stats.pearsonr(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values)[0])
        pcc_list = np.array(pcc_list)
        pcc_list[grid_b.sum(1) == 0] = np.inf
        pcc_list[pd.isna(pd.Series(pcc_list))] = 0
        adata_list[i].uns['pcc'] = pcc_list
    pcc_compare = []
    for i in range(1, len(adata_list)):
        pcc_compare.append([adata_list[i].uns['pcc'][adata_list[i].uns['pcc'] != np.inf].mean(), i, method])
    pcc_compare = pd.DataFrame(pcc_compare).fillna(0)
    pcc_compare.columns = ['pcc', 'slice', 'method']
    print(pcc_compare)
    
    
    ### save
    output_dir = 'G:/dataset/1_DLPFC/output/'
    file = open(output_dir + 'Donor3_spatial_index.txt', 'a') 
    file.write('\n')
    file.write(method + ':\n')
    file.write('origin:' + str(adatas.uns['scc_origin']) +'\n')
    file.write('aligned:' + str(adatas.uns['scc_aligned']) + '\n')
    file.write('\n')

    for i in range(4):
        batch = list(groups.keys())[i].split('_')[1]
        for k,v in adata_list[i].uns['moranI'].items():
            file.write(method +', moranI, '+str(k)+', '+str(v)+', '+batch+'\n')
        for k,v in adata_list[i].uns['moranI_align'].items():
            file.write(method +', moranI_align, '+str(k)+', '+str(v)+', '+batch+'\n')
        for k,v in adata_list[i].uns['gearyC'].items():
            file.write(method +', gearyC, '+str(k)+', '+str(v)+', '+batch+'\n')
        for k,v in adata_list[i].uns['gearyC_align'].items():
            file.write(method +', gearyC_align, '+str(k)+', '+str(v)+', '+batch+'\n')

    file.write('\n')
    # 注意关闭文件
    file.close()

    ssim_compare.to_csv(output_dir + 'Donor3_spatial_index.txt',mode='a', index=True, header=True)
    pcc_compare.to_csv(output_dir + 'Donor3_spatial_index.txt',mode='a', index=True, header=True)

D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn

2.4385943714739793
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2.2665130327002347
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000


C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:185: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_a = grid_a.loc[:, col]
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:186: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_b = grid_b.loc[:, col]
D:\bio\metirc\scib-main\index\spatial\SSIM.py:36: RuntimeWarning: invalid value encountered in divide
  im1, im2 = im1 / im1.max(), im2 / im2.max()
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:185: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_a = grid_a.loc[:, col]
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:186: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_b = grid_b.loc[:, col]
D:\b

       ssim  slice  method
0  0.943720      1  SPACEL
1  0.906881      2  SPACEL
2  0.918925      3  SPACEL
        pcc  slice  method
0  0.959204      1  SPACEL
1  0.923520      2  SPACEL
2  0.936749      3  SPACEL


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn

2.380483607738061
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2.336646571119643
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000
1
10
50
100
500
1000


C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:185: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_a = grid_a.loc[:, col]
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:186: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_b = grid_b.loc[:, col]
D:\bio\metirc\scib-main\index\spatial\SSIM.py:36: RuntimeWarning: invalid value encountered in divide
  im1, im2 = im1 / im1.max(), im2 / im2.max()
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:185: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_a = grid_a.loc[:, col]
C:\Users\23147\AppData\Local\Temp\ipykernel_11440\3181888351.py:186: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  grid_b = grid_b.loc[:, col]
D:\b

       ssim  slice  method
0  0.930609      1  SPIRAL
1  0.913353      2  SPIRAL
2  0.901520      3  SPIRAL
        pcc  slice  method
0  0.951667      1  SPIRAL
1  0.932859      2  SPIRAL
2  0.921592      3  SPIRAL


D:\anaconda\envs\index\lib\site-packages\scipy\stats\_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [6]:
groups = adatas.obs.groupby("batch").indices
groups

{'D2_151669': array([   0,    1,    2, ..., 3658, 3659, 3660], dtype=int64),
 'D2_151670': array([3661, 3662, 3663, ..., 7156, 7157, 7158], dtype=int64),
 'D2_151671': array([ 7159,  7160,  7161, ..., 11266, 11267, 11268], dtype=int64)}

In [4]:
adata_list = []
for i in range(151507,151511):
    print(i)

    input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

    adata_temp = sc.read_visium(input_dir)
    adata_temp.var_names_make_unique()
    adata_temp.obs_names_make_unique()
    adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
    adata_temp.obs['batch'] = adata_label['batch'].astype("category")
    adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
    print(adata_temp)
    adata_list.append(adata_temp)

adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
print(adata_pre)

del adata_temp
del adata_list

151507


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151508


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4384 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151509


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4789 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
151510


D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\anaconda\envs\index\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4634 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'
AnnData object with n_obs × n_vars = 18033 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'batch', 'ground.truth'
    obsm: 'spatial'


In [5]:
method = 'SPIRAL'

In [ ]:
# # STUtility coor  (17889, 8)
# STUtility_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/STUtility/Donor1-STutil-coords.csv", sep=" ")
# print(STUtility_coor_pd.shape)
# print(STUtility_coor_pd.head())

# STUtility_coord_list = []
# STUtility_coor_pd.rename(index=lambda s: s.split('_',1)[0] + '-' + str(int(s.split('_',1)[1]) + 151506), inplace=True)

# adatas = adata_pre[STUtility_coor_pd.index.tolist(), :]
# adatas.obsm['spatial_aligned'] = STUtility_coor_pd[['align_x', 'align_y']].values
# adatas.obsm['spatial_origin'] = STUtility_coor_pd[['x', 'y']].values
# print(adatas)

In [ ]:
# # moscot  (18033, 2)
# # moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_affine_donor1.txt", header = None)
# moscot_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/moscot/moscot_warp_donor1.txt", header = None)
# print(moscot_coor_pd.shape)
# print(moscot_coor_pd.head())

# adatas = adata_pre

# adatas.obsm['spatial_aligned'] = moscot_coor_pd.values
# adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()

# print(adatas)

In [ ]:
# # STalign 
# adata_list = []
# for i in range(151507,151511):
#     print(i)

#     input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

#     adata_temp = sc.read_visium(input_dir)
#     adata_temp.var_names_make_unique()
#     adata_temp.obs_names_make_unique()
#     adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
#     adata_temp.obs['batch'] = adata_label['batch'].astype("category")
#     adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
#     if i != 151510:
#         STalign_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/STalign/'+str(i)+'_aligned_to_'+str(i+1)+'.csv')
#         STalign_coor_pd.index = STalign_coor_pd['0']
#         adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#         adata_temp.obsm['spatial_aligned'] = STalign_coor_pd[['aligned_y', 'aligned_x']].values
#     else:
#         adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#         adata_temp.obsm['spatial_aligned'] = adata_temp.obsm['spatial'].copy()
    
#     print(adata_temp)
#     adata_list.append(adata_temp)

# adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
# print(adata_pre)
# adatas = adata_pre

# del adata_temp
# del adata_list

In [ ]:
# # paste  17978 × 33538
# adata_list = []
# for i in range(151507,151511):
#     print(i)

#     input_dir = 'G:/dataset/1_DLPFC/input/'+str(i)+'/'  # Replace it with your file path

#     adata_temp = sc.read_visium(input_dir)
#     adata_temp.var_names_make_unique()
#     adata_temp.obs_names_make_unique()
#     adata_label = pd.read_csv(input_dir + 'truth.csv', index_col=0)
#     adata_temp.obs['batch'] = adata_label['batch'].astype("category")
#     adata_temp.obs['ground.truth'] = adata_label['ground.truth'].astype("category")
    
#     paste_coor_pd = pd.read_csv('G:/dataset/1_DLPFC/output/PASTE/'+str(i)+'.csv')
#     paste_coor_pd.index = paste_coor_pd['spot']
#     paste_coor_pd.rename(index=lambda s: s.split('.',1)[0], inplace=True)
#     adata_temp = adata_temp[paste_coor_pd.index.tolist(), :]
#     adata_temp.obsm['spatial_origin'] = adata_temp.obsm['spatial'].copy()
#     adata_temp.obsm['spatial_aligned'] = paste_coor_pd[['aligned_x', 'aligned_y']].values
    
#     print(adata_temp)
#     adata_list.append(adata_temp)

# adata_pre = ad.concat(adata_list, index_unique="-", keys = [i for i in range(151507,151511)])
# print(adata_pre)
# adatas = adata_pre

# del adata_temp
# del adata_list

In [ ]:
# # SPACEL  (18033, 3)
# SPACEL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPACEL/donor1_aligned_coordinates.csv")
# print(SPACEL_coor_pd.shape)
# print(SPACEL_coor_pd.head())

# adatas = adata_pre

# adatas.obsm['spatial_aligned'] = SPACEL_coor_pd[['X','Y']].values
# adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()

# print(adatas)

In [6]:
SPIRAL_coor_pd = pd.read_csv("G:/dataset/1_DLPFC/output/SPIRAL/output/spiral/new_coord_151507_151508_151509_151510_modify.csv")
print(SPIRAL_coor_pd.head())
print(SPIRAL_coor_pd.shape)
print(adata_pre.obs_names[:5])

SPIRAL_coor_pd.index = SPIRAL_coor_pd['Unnamed: 0']
SPIRAL_coor_pd.rename(index=lambda s: s.split('-',1)[1] + '-' + s.split('-',1)[0], inplace=True)
print(SPIRAL_coor_pd.head())

adatas = adata_pre[SPIRAL_coor_pd.index.tolist(), :]

adatas.obsm['spatial_aligned'] = SPIRAL_coor_pd[['x', 'y']].values
adatas.obsm['spatial_origin'] = adatas.obsm['spatial'].copy()
print(adatas)

                  Unnamed: 0       x       y  clusters   batch
0  151507-AAACAACGAATAGTTC-1  3276.0  2514.0         3  151507
1  151507-AAACAAGTATCTCCCA-1  9178.0  8520.0         4  151507
2  151507-AAACAATCTACTAGCA-1  5133.0  2878.0         3  151507
3  151507-AAACACCAATAACTGC-1  3462.0  9581.0         6  151507
4  151507-AAACAGCTTTCAGAAG-1  2779.0  7663.0         2  151507
(17985, 5)
Index(['AAACAACGAATAGTTC-1-151507', 'AAACAAGTATCTCCCA-1-151507',
       'AAACAATCTACTAGCA-1-151507', 'AAACACCAATAACTGC-1-151507',
       'AAACAGCTTTCAGAAG-1-151507'],
      dtype='object')
                                          Unnamed: 0       x       y  \
Unnamed: 0                                                             
AAACAACGAATAGTTC-1-151507  151507-AAACAACGAATAGTTC-1  3276.0  2514.0   
AAACAAGTATCTCCCA-1-151507  151507-AAACAAGTATCTCCCA-1  9178.0  8520.0   
AAACAATCTACTAGCA-1-151507  151507-AAACAATCTACTAGCA-1  5133.0  2878.0   
AAACACCAATAACTGC-1-151507  151507-AAACACCAATAACTGC-1  3462.0  

# Pre Exam

In [7]:
print(adatas.obsm['spatial_origin'].shape)
print(adatas.obsm['spatial_aligned'].shape)
print(adatas.obsm['spatial_origin'])
print(adatas.obsm['spatial_aligned'])

(17985, 2)
(17985, 2)
[[ 3276  2514]
 [ 9178  8520]
 [ 5133  2878]
 ...
 [ 3514  5649]
 [10564  6907]
 [10289  6664]]
[[ 3276.          2514.        ]
 [ 9178.          8520.        ]
 [ 5133.          2878.        ]
 ...
 [ 4315.25287356  3235.56704981]
 [10671.57471264  3790.93103448]
 [10363.          3613.        ]]


# SCC

In [ ]:
# source
adatas.obsm['spatial'] = adatas.obsm['spatial_origin'].copy()
knn = 6
g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
scs = index.spatial_coherence_score(g, node_dict, 100)
adatas.uns['scc_origin'] = scs
print(scs)

# align
adatas.obsm['spatial'] = adatas.obsm['spatial_aligned'].copy()
knn = 6
g, node_dict = index.generate_graph_from_labels(adatas, adatas.obs['ground.truth'] ,knn)
scs = index.spatial_coherence_score(g, node_dict, 100)
adatas.uns['scc_aligned'] = scs
print(scs)

In [ ]:
print(adatas)

# Moran's I & Geary's C

In [ ]:
groups = adatas.obs.groupby("batch").indices
adata_list = [adatas[i] for i in groups.values()]

for i in range(4):
    batch = list(groups.keys())[i].split('_')[1]
    print(batch)
    input_dir = 'G:/dataset/1_DLPFC/input/' + batch + '/'
    SVGs = pd.read_csv(input_dir + 'SPARKX.csv', index_col=0)
    print(SVGs.shape)
    SVGs = SVGs.sort_values(by="adjustedPval",ascending=True)
    
    # source
    moranI_dict = {}
    gearyC_dict = {}
    sc.pp.neighbors(adata_list[i], use_rep='spatial_origin')
    num = [1,10,50,100,500,1000]
    for SVGnum in num:
        print(SVGnum)
        if SVGnum > SVGs.shape[0]:
            break
        moranI = np.mean( sc.metrics.morans_i(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
        moranI_dict["moranI_"+str(SVGnum)] = moranI
        gearyC = np.mean( sc.metrics.gearys_c(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
        gearyC_dict["gearyC_"+str(SVGnum)] = gearyC
        
    # align
    moranI_align_dict = {}
    gearyC_align_dict = {}
    sc.pp.neighbors(adata_list[i], use_rep='spatial_aligned')
    num = [1,10,50,100,500,1000]
    for SVGnum in num:
        print(SVGnum)
        if SVGnum > SVGs.shape[0]:
            break
        moranI = np.mean( sc.metrics.morans_i(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
        moranI_align_dict["moranI_"+str(SVGnum)] = moranI
        gearyC = np.mean( sc.metrics.gearys_c(adata_list[i], vals = adata_list[i][:, SVGs.index.tolist()[:SVGnum]].X.toarray().T) )
        gearyC_align_dict["gearyC_"+str(SVGnum)] = gearyC
        
    adata_list[i].uns['moranI'] = moranI_dict
    adata_list[i].uns['moranI_align'] = moranI_align_dict
    adata_list[i].uns['gearyC'] = gearyC_dict
    adata_list[i].uns['gearyC_align'] = gearyC_align_dict
    print(adata_list[i])



In [ ]:
for i in range(4):
    batch = list(groups.keys())[i].split('_')[1]
    print(batch)
    print('moranI')
    print(adata_list[i].uns['moranI'])
    print('moranI_align')
    print(adata_list[i].uns['moranI_align'])
    print('gearyC')
    print(adata_list[i].uns['gearyC'])
    print('gearyC_align')
    print(adata_list[i].uns['gearyC_align'])

# SSIM

In [8]:
groups = adatas.obs.groupby("batch").indices
adata_list = [adatas[i] for i in groups.values()]

for i in range(len(adata_list)):
    adata_list[i].obsm['spatial_aligned'] = pd.DataFrame(adata_list[i].obsm['spatial_aligned'], index=adata_list[i].obs_names)
    adata_list[i].obsm['spatial_aligned'].columns = ['X', 'Y']

# each slice is rasterized on the overlap of the previous slice
for i in range(1, len(adata_list)):
    x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i - 1].obsm['spatial_aligned'].max().X]))
    y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i - 1].obsm['spatial_aligned'].max().Y]))
    x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i - 1].obsm['spatial_aligned'].min().X]))
    y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i - 1].obsm['spatial_aligned'].min().Y]))
    adata_list[i].uns['grid_prop_withbefore'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                         grid_max=[x_max, y_max], grid_min=[x_min, y_min])

# each slice is rasterized on the overlap of the next slice
for i in range(0, len(adata_list)-1):
    x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i + 1].obsm['spatial_aligned'].max().X]))
    y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i + 1].obsm['spatial_aligned'].max().Y]))
    x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i + 1].obsm['spatial_aligned'].min().X]))
    y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i + 1].obsm['spatial_aligned'].min().Y]))
    adata_list[i].uns['grid_prop_withnext'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                       grid_max=[x_max, y_max], grid_min=[x_min, y_min])

In [11]:
print(adata_list[2].uns['grid_prop_withnext'])
print(adata_list[3].uns['grid_prop_withbefore'])

     Layer 1   Layer 2   Layer 3   Layer 4   Layer 5   Layer 6   WM
0   0.100000  0.100000  0.800000  0.000000  0.000000  0.000000  0.0
1   0.115385  0.134615  0.711538  0.000000  0.019231  0.019231  0.0
2   0.345455  0.109091  0.163636  0.000000  0.072727  0.309091  0.0
3   0.000000  0.000000  0.609756  0.073171  0.048780  0.268293  0.0
4   0.000000  0.000000  0.187500  0.500000  0.281250  0.031250  0.0
..       ...       ...       ...       ...       ...       ...  ...
95  0.032258  0.322581  0.645161  0.000000  0.000000  0.000000  0.0
96  0.000000  0.058824  0.941176  0.000000  0.000000  0.000000  0.0
97  0.000000  0.052632  0.947368  0.000000  0.000000  0.000000  0.0
98  0.000000  0.285714  0.714286  0.000000  0.000000  0.000000  0.0
99  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0

[100 rows x 7 columns]
     Layer 1   Layer 2   Layer 3   Layer 4   Layer 5   Layer 6   WM
0   0.111111  0.000000  0.888889  0.000000  0.000000  0.000000  0.0
1   0.102041  0.163265  

In [ ]:
# calculate SSIM on the overlap of each two adjacent slices
for i in range(1, len(adata_list)):
    grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
    grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
    col = set(grid_a.columns) & set(grid_b.columns)
    grid_a = grid_a.loc[:, col]
    grid_b = grid_b.loc[:, col]
    ssim_list = []
    for n in range(grid_a.shape[0]):
        ssim_list.append(index.ssim(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values))
    ssim_list = np.array(ssim_list)
    ssim_list[grid_b.sum(1) == 0] = np.inf
    ssim_list[pd.isna(pd.Series(ssim_list))] = 0
    adata_list[i].uns['ssim'] = ssim_list

ssim_compare = []
for i in range(1, len(adata_list)):
    ssim_compare.append([adata_list[i].uns['ssim'][adata_list[i].uns['ssim'] != np.inf].mean(), i, method])
    # ssim_compare.append([paste[i].uns['ssim'][paste[i].uns['ssim'] != np.inf].mean(), i, 'paste'])

ssim_compare = pd.DataFrame(ssim_compare).fillna(0)
ssim_compare.columns = ['ssim', 'slice', 'method']
# ssim_compare.to_csv('G:/dataset/1_DLPFC/output/ssim_compare_donor1.csv')
print(ssim_compare)

# PCC

In [ ]:
import scipy
# groups = adatas.obs.groupby("batch").indices
# adata_list = [adatas[i] for i in groups.values()]

for i in range(len(adata_list)):
    adata_list[i].obsm['spatial_aligned'] = pd.DataFrame(adata_list[i].obsm['spatial_aligned'], index=adata_list[i].obs_names)
    adata_list[i].obsm['spatial_aligned'].columns = ['X', 'Y']

# each slice is rasterized on the overlap of the previous slice
for i in range(1, len(adata_list)):
    x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i - 1].obsm['spatial_aligned'].max().X]))
    y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i - 1].obsm['spatial_aligned'].max().Y]))
    x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i - 1].obsm['spatial_aligned'].min().X]))
    y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i - 1].obsm['spatial_aligned'].min().Y]))
    adata_list[i].uns['grid_prop_withbefore'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                         grid_max=[x_max, y_max], grid_min=[x_min, y_min])

# each slice is rasterized on the overlap of the next slice
for i in range(0, len(adata_list)-1):
    x_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().X, adata_list[i + 1].obsm['spatial_aligned'].max().X]))
    y_max = np.ceil(np.min([adata_list[i].obsm['spatial_aligned'].max().Y, adata_list[i + 1].obsm['spatial_aligned'].max().Y]))
    x_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().X, adata_list[i + 1].obsm['spatial_aligned'].min().X]))
    y_min = np.floor(np.max([adata_list[i].obsm['spatial_aligned'].min().Y, adata_list[i + 1].obsm['spatial_aligned'].min().Y]))
    adata_list[i].uns['grid_prop_withnext'] = index.cal_grid_prop(adata_list[i], spatial_key='spatial_aligned', celltype_key='ground.truth',
                                                       grid_max=[x_max, y_max], grid_min=[x_min, y_min])
    
# calculate PCC on the overlap of each two adjacent slices
for i in range(1, len(adata_list)):
    grid_a = adata_list[i].uns['grid_prop_withbefore'].copy()
    grid_b = adata_list[i - 1].uns['grid_prop_withnext'].copy()
    col = set(grid_a.columns) & set(grid_b.columns)
    grid_a = grid_a.loc[:, col]
    grid_b = grid_b.loc[:, col]
    pcc_list = []
    for n in range(grid_a.shape[0]):
        pcc_list.append(scipy.stats.pearsonr(grid_a.iloc[n, :].values, grid_b.iloc[n, :].values)[0])
    pcc_list = np.array(pcc_list)
    pcc_list[grid_b.sum(1) == 0] = np.inf
    pcc_list[pd.isna(pd.Series(pcc_list))] = 0
    adata_list[i].uns['pcc'] = pcc_list

pcc_compare = []
for i in range(1, len(adata_list)):
    pcc_compare.append([adata_list[i].uns['pcc'][adata_list[i].uns['pcc'] != np.inf].mean(), i, method])
    # ssim_compare.append([adata_list[i].uns['ssim'][adata_list[i].uns['ssim'] != np.inf].mean(), i, 'STUtility'])

pcc_compare = pd.DataFrame(pcc_compare).fillna(0)
pcc_compare.columns = ['pcc', 'slice', 'method']
print(pcc_compare)
# pcc_compare.to_csv('G:/dataset/1_DLPFC/output/pcc_compare_donor1.csv')

In [ ]:
output_dir = 'G:/dataset/1_DLPFC/output/'
file = open(output_dir + 'Donor1_spatial_index.txt', 'a') 

file.write(method + ':\n')
file.write('origin:' + str(adatas.uns['scc_origin']) +'\n')
file.write('aligned:' + str(adatas.uns['scc_aligned']) + '\n')
file.write('\n')

for i in range(4):
    batch = list(groups.keys())[i].split('_')[1]
    for k,v in adata_list[i].uns['moranI'].items():
        file.write(method +', moranI, '+str(k)+', '+str(v)+', '+batch+'\n')
    for k,v in adata_list[i].uns['moranI_align'].items():
        file.write(method +', moranI_align, '+str(k)+', '+str(v)+', '+batch+'\n')
    for k,v in adata_list[i].uns['gearyC'].items():
        file.write(method +', gearyC, '+str(k)+', '+str(v)+', '+batch+'\n')
    for k,v in adata_list[i].uns['gearyC_align'].items():
        file.write(method +', gearyC_align, '+str(k)+', '+str(v)+', '+batch+'\n')
    
file.write('\n')
# 注意关闭文件
file.close()

ssim_compare.to_csv(output_dir + 'Donor1_spatial_index.txt',mode='a', index=True, header=True)
pcc_compare.to_csv(output_dir + 'Donor1_spatial_index.txt',mode='a', index=True, header=True)